# Saskia's Shady Sudoku

In de community challenge van deze week wordt gevraagd om een script te schrijven om een ingevulde Sudoku te controleren op juistheid.

## Spelregels Sudoku

Het spel Sudoku bestaat uit een in blokjes opgedeeld speelveld (9x9) waarin de nummers 1 tot en met 9 worden ingevuld. Deze mogen niet 2x op dezelfde horizontale of verticale as voorkomen. 

Daarna is het speelveld onderverdeeld in blokken van 3x3. Hierin mogen ook geen dubbele getallen voorkomen.

Verder is er een variant, genaamd X-Sudoku, waarbij dubbele getallen op de twee diagonale assen ook verboden zijn.

## Mijn oplossing

Doordat het speelveld in blokken is verdeeld, doet me dit denken aan een kaart met coordinaten. Via de coordinaten kan je gemakkelijk een vorm of volgorde creeren, zoals voor de blokken van 3 en de diagonale assen.

Daarnaast moet een reeks getallen altijd uit dezelfde getallen bestaan, waardoor de som altijd 45 is. Ik weet nog niet of dit gebruikt kan worden, het is wel een snelle en makkelijke methode om te controleren of een reeks klopt, maar het is ook foutgevoelig. Zo is 5x9 ook 45.

Ik heb gekozen om de pandas module te gebruiken, omdat het coordinaten systeem hier eigenlijk al inzit, met de kolommen en rijen als coordinaten. Daarnaast is het makkelijk om via pandas het JSON formaat van de sudoku in te lezen.

De methode hoe we de getallen uit het dataframe halen is voor elke spelregel dat we controleren anders, maar uiteindelijk krijgen we een reeks aan getallen waarbij gekeken moet worden of er geen duplicates inzitten.

Nadat we de data hebben ingelezen, beginnen we met een functie die de controle op duplicates uitvoerd.

Vervolgens kunnen we voor elke spelregel de code schrijven die de waardes in dat deel van het veld verzameld en deze naar de controle functie sturen.

In [1]:
import requests
import pandas as pd

url = "https://community-challenge.netlify.app/data/sudoku.json"
r = requests.get(url)
json_file = r.json()

sudoku = json_file["sudoku"]

# hier heb ik een aantal getallen veranderd om te kunnen controleren of dubbele getallen correct worden herkend.
sudoku2 = [[2, 5, 6, 8, 9, 1, 4, 7, 3],
 [2, 4, 9, 7, 2, 6, 8, 5, 1],
 [2, 7, 1, 3, 4, 5, 9, 2, 6],
 [6, 8, 3, 2, 5, 4, 1, 9, 7],
 [9, 1, 5, 6, 8, 7, 3, 4, 2],
 [4, 2, 7, 9, 1, 3, 5, 6, 8],
 [7, 6, 4, 5, 3, 8, 2, 1, 9],
 [1, 3, 2, 4, 7, 9, 6, 5, 5],
 [5, 9, 8, 1, 6, 6, 7, 3, 4]]

# Name columns by letters and name indexes so the df can be transposed
column_names = [letter for letter in "abcdefghi"]
index_names = [number for number in range(9)]
    
df = pd.DataFrame(sudoku, columns=column_names, index=index_names)

### Horizontale en verticale assen

Dit was vrij eenvoudig, in pandas kan je per kolom filteren, alle waardes naar een lijst sturen. Deze lijst wordt door de controle functie gecontroleerd. Om aan te geven waar de fout zit, stoppen we de coordinaten in een dictionary.

Voor de tweede as, transposen we het dataframe waardoor we de index als kolommen kunnen gebruiken. Hierdoor kunnen we dezelfde functie oproepene met het getransposede dataframe. 

In [2]:
def find_duplicates(numbers_list):
    """Find and return list of duplicates in a list"""
    return [number for number in numbers_list if numbers_list.count(number) > 1]

def check_column_axis(df):
    """Check column axis for duplicates"""
    column_dupli_coords = {}
    for column in df.columns:
        dupl_num = list(set(find_duplicates(list(df[column].values))))
        if len(dupl_num) >= 1:
            coords = df.index[df[column] == dupl_num[0]].tolist()
            column_dupli_coords[column] = coords
            return column_dupli_coords

def check_index_axis(df):
    """Check Index axis for duplicates by transposing index to columns and sent to column function"""
    df = df.T
    return check_column_axis(df)

def check_both_axis(df):
    if check_column_axis(df) != None:
        print(f"Gevonden duplicaten op verticale as: {check_column_axis(df)}")
    if check_index_axis(df) != None:
        print(f"Gevonden duplicaten op horizontale as: {check_index_axis(df)}")


### Parts/blokken

Om de gehele sudoku in blokken van 3x3 te verdelen, heb ik een loop gemaakt waarbij de range functie stappen van 3 maakt. Dit zijn de index-slices die elk blok uit het dataframe filteren. 

Alle slices hebben we in een lijst opgeslagen zodat we daar weer doorheen kunnen loopen.

Nadat we een blok/part uit het gehele dataframe hebben gefilterd, kunnen we een lijst maken met alle waardes van dat blok. Omdat dit voor elk blok gebeurd heb ik hiervoor ook een functie aangemaakt.

Als er dubbele getallen zijn gevonden, dan worden de "coordinaten" van dat onderdeel opgeslagen en nadat elk blok is gecontroleerd worden alle blokken met duplicates uitgeprint.

In [3]:
def convert_slice_to_list(slice):
    """Convert all values in a slice to a list to check for duplicates"""
    list_of_rows = slice.values.tolist()
    slice_list = []
    for i in range(3):
        for j in range(3):
            slice_list.append(list_of_rows[i][j])
    return slice_list

def check_parts(df):
    """
    Divide whole map in 9 3x3 "Parts", define coordinates/slices for each part.
    Extract values for each part, check values for duplicates and save coordinates if there are duplicates.
    """
    
    all_part_slices = []
    for i in range(0, 7, 3):
        for j in range(0, 7, 3):
            part_slice = slice(i, i+3), slice(j, j+3)
            all_part_slices.append(part_slice)
        
    dupl_part_coordinates =[]
    for part in all_part_slices:
        part_dupl = find_duplicates(convert_slice_to_list(df.iloc[part]))
        if len(part_dupl) >= 1:
            dupl_part_coordinates.append(part)
    for coord in dupl_part_coordinates:
        print("\nDubbele getallen gevonden in blok:")
        print(df.iloc[coord])

### Diagonale assen

Om de coordinaten/index van de diagonale assen te krijgen, heb ik een vergelijkbare logica als bij de blokken gebruikt.

Het begintpunt linksboven is 0, 0. Rechtsonder is 8, 8. Beide getallen van elke index zijn gelijk, maar moeten voor elke cell/stap 1 omhoog.

Het beginpunt voor de andere as is 0, 8 en het eindpunt is 8, 0. Voor elke cell hiervan moet bij het ene getal 1 erbij, terwijl bij de andere 1 eraf gaat.

In [4]:
def check_diagonal_axis(df):
    """Get indexes for both diagonal axis and then check their values for duplicates"""

    diagonal1_values = []
    diagonal2_values = []

    for i in range(9):
        cell_slice = i, i
        diagonal1_values.append(df.iloc[cell_slice])

    for i in range(9):
        x, y = 8, 0
        cell_slice = x-i, y+i
        diagonal2_values.append(df.iloc[cell_slice])      
            
    if len(find_duplicates(diagonal1_values)) >= 1 and len(find_duplicates(diagonal2_values)) >= 1:
        print("Dubbele getallen op beide diagonale assen gevonden:")
        print(f"A0 - I8: {find_duplicates(diagonal1_values)}")
        print(f"A8 - I0: {find_duplicates(diagonal2_values)}")
    elif len(diagonal1_values) >= 1:
        print("Dubbele getallen op de diagonale as van A0 naar I8 gevonden")
        print(f"A0 - I8: {find_duplicates(diagonal1_values)}")
    elif len(diagonal2_values) >= 1:
        print("Dubbele getallen op de diagonale as van A8 naar I0 gevonden")
        print(f"A8 - I0: {find_duplicates(diagonal2_values)}")

En gebruiken we nu alle functies op het dataframe/de sudoku.

In [5]:
check_both_axis(df)
check_parts(df)
check_diagonal_axis(df)

Dubbele getallen op beide diagonale assen gevonden:
A0 - I8: [2, 4, 2, 8, 2, 8, 4]
A8 - I0: [5, 3, 4, 9, 4, 9, 5, 3]


Hieronder het dataframe ter controle:

In [6]:
df

,a,b,c,d,e,f,g,h,i
0,2,5,6,8,9,1,4,7,3
1,3,4,9,7,2,6,8,5,1
2,8,7,1,3,4,5,9,2,6
3,6,8,3,2,5,4,1,9,7
4,9,1,5,6,8,7,3,4,2
5,4,2,7,9,1,3,5,6,8
6,7,6,4,5,3,8,2,1,9
7,1,3,2,4,7,9,6,8,5
8,5,9,8,1,6,2,7,3,4
